In [23]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

In [25]:
df = pd.read_excel('Assignment_Conjoint Analysis.xlsx', sheet_name= 'RawData', header = 0)
# drop nulls
df = df.dropna(how = 'all')
# rename cols to make it easier. 
column_names = [
    "Small / All Movies / Mall",
    "Medium / All Movies / Mall",
    "Large / All Movies / Mall",
    "Small / Some Movies / Mall",
    "Medium / Some Movies / Mall",
    "Large / Some Movies / Mall",
    "Small / Popular Movies / Mall",
    "Medium / Popular Movies / Mall",
    "Large / Popular Movies / Mall",
    "Small / All Movies / Plaza",
    "Medium / All Movies / Plaza",
    "Large / All Movies / Plaza",
    "Small / Some Movies / Plaza",
    "Medium / Some Movies / Plaza",
    "Large / Some Movies / Plaza",
    "Small / Popular Movies / Plaza",
    "Medium / Popular Movies / Plaza",
    "Large / Popular Movies / Plaza",
    "Small / All Movies / Standalone",
    "Medium / All Movies / Standalone",
    "Large / All Movies / Standalone",
    "Small / Some Movies / Standalone",
    "Medium / Some Movies / Standalone",
    "Large / Some Movies / Standalone",
    "Small / Popular Movies / Standalone",
    "Medium / Popular Movies / Standalone",
    "Large / Popular Movies / Standalone",
]
df.columns = column_names
# reset idx to start w 0
df.reset_index(drop = True, inplace = True)
# melt the df by stacking cols to make long format
long_df = pd.melt(df.iloc[1:], var_name = 'Attributes', value_name = 'Ratings')
long_df[['Size', 'Movie Selection', 'Location']] = long_df['Attributes'].str.split(' / ', expand=True)
long_df = long_df.drop(columns = 'Attributes')

long_df = pd.get_dummies(long_df, columns = ['Size', 'Movie Selection', 'Location'], drop_first=True)

# converting Rating to numeric since it was causing issues while treated as obj
long_df['Ratings'] = pd.to_numeric(long_df['Ratings'], errors='coerce')
long_df = long_df.dropna(subset=['Ratings'])

for col in long_df.columns:
    if long_df[col].dtype == 'object':
        long_df[col] = pd.to_numeric(long_df[col], errors='coerce')
#saving to csv so I can mess around with it in R and apply the logic here.
saved =df.to_csv('long_df.csv')


In [26]:


df = pd.read_csv('long_df.csv')

print(df.info())
print("\nFirst few rows of the dataframe:")
print(df.head())

df_melted = df.melt(id_vars=['Unnamed: 0'], var_name='attribute_level', value_name='rating')
df_melted['respondent'] = df_melted['Unnamed: 0']
df_melted = df_melted.drop('Unnamed: 0', axis=1)

df_melted[['Size', 'Movies', 'Location']] = df_melted['attribute_level'].str.split(' / ', expand=True)
df_melted = df_melted.drop('attribute_level', axis=1)

df_melted['rating'] = pd.to_numeric(df_melted['rating'], errors='coerce')

df_melted = df_melted.dropna(subset=['rating'])

df_dummies = pd.get_dummies(df_melted, columns=['Size', 'Movies', 'Location'], drop_first=True)

df_dummies.columns = df_dummies.columns.str.replace(' ', '_').str.replace('/', '_').str.replace('(', '').str.replace(')', '')

print("\nColumn names and data types:")
print(df_dummies.dtypes)

size_vars = [col for col in df_dummies.columns if col.startswith('Size_')]
movies_vars = [col for col in df_dummies.columns if col.startswith('Movies_')]
location_vars = [col for col in df_dummies.columns if col.startswith('Location_')]

formula = 'rating ~ ' + ' + '.join(size_vars + movies_vars + location_vars)
print("\nRegression formula:")
print(formula)

model = ols(formula, data=df_dummies).fit()
coefficients = model.params

part_worth = {
    'Size': {level.split('_')[1]: coef for level, coef in coefficients.items() if level.startswith('Size_')},
    'Movies': {level.split('_')[1]: coef for level, coef in coefficients.items() if level.startswith('Movies_')},
    'Location': {level.split('_')[1]: coef for level, coef in coefficients.items() if level.startswith('Location_')}
}

for attribute in part_worth:
    base_level_utility = -sum(part_worth[attribute].values())
    part_worth[attribute][f"{attribute}_Base"] = base_level_utility

importance = {}
for attribute, levels in part_worth.items():
    importance[attribute] = (max(levels.values()) - min(levels.values())) / sum((max(levels.values()) - min(levels.values())) for levels in part_worth.values())

print("\nPart-worth utilities:")
for attribute, levels in part_worth.items():
    print(f"\n{attribute}:")
    for level, utility in levels.items():
        print(f"  {level}: {utility:.4f}")

print("\nAttribute Importance:")
for attribute, imp in importance.items():
    print(f"{attribute}: {imp*100:.2f}%")

for attribute, levels in part_worth.items():
    most_preferred = max(levels, key=levels.get)
    least_preferred = min(levels, key=levels.get)
    print(f"\n{attribute}:")
    print(f"  Most preferred: {most_preferred} ({levels[most_preferred]:.4f})")
    print(f"  Least preferred: {least_preferred} ({levels[least_preferred]:.4f})")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Unnamed: 0                            22 non-null     int64 
 1   Small / All Movies / Mall             22 non-null     object
 2   Medium / All Movies / Mall            22 non-null     object
 3   Large / All Movies / Mall             22 non-null     object
 4   Small / Some Movies / Mall            22 non-null     object
 5   Medium / Some Movies / Mall           22 non-null     object
 6   Large / Some Movies / Mall            22 non-null     object
 7   Small / Popular Movies / Mall         22 non-null     object
 8   Medium / Popular Movies / Mall        22 non-null     object
 9   Large / Popular Movies / Mall         22 non-null     object
 10  Small / All Movies / Plaza            22 non-null     object
 11  Medium / All Movies / Plaza       